In [1]:
using Distributions, Random, Plots, SpecialFunctions
Random.seed!(42)

TaskLocalRNG()

## Problem 1

For a large sample, given $Var(F) = \sigma^2$

In case of i.i.d draws, $Var(\bar{X}) = \frac{\sigma^2}{n}$

In case of a Markov chain with F as the stationary distribution [Reference, pg.1](https://arxiv.org/pdf/math/0409112.pdf)

$Var(\bar{Y}) = \frac{\sigma^2}{n} + 2 \sum_{i=1} cov_F(Y_1, Y_1+i)$

$\because$ The covariance term can be any real number, we cannot say if the which estimator has a larger variance.

## Problem 2

a. The joint posterior distribution is 

$$q \propto \prod_{i} \frac{1}{\sqrt{2\pi}} e^{\frac{-(y_i-\mu)^2}{2}} \frac{1}{\sqrt{\nu\pi}} \frac{\Gamma(\frac{\nu+1}{2})}{\Gamma(\frac{\nu}{2})} (1 + \frac{\mu^2}{\nu})^{\frac{-\nu + 1}{2}} \frac{\nu^{a_o-1}e^{-b_o\nu}{b_o}^{a_o}}{\Gamma(a_o)} \text{ for } 2 \le \nu \le \infty$$

In [2]:
function generate(a, b, n)
    N = rand(Truncated(Gamma(a, b),2, Inf), n)
    M = rand(Normal(rand(TDist(mean(N))), 1), n)
    return N, M
end

generate (generic function with 1 method)

In [3]:
function q_tilde(a_o, b_o, n, μ, ν, y)
    if (ν < 2) 
        return 0
    end
    x = (1+μ^2/ν)^((-ν+1)/2)*ν^(a_o-1)ℯ^(-b_o*ν) * gamma((ν+1)/2)/gamma(ν/2)
    for i in 1:n
        x = x * ℯ^(-((y[i]-μ)/2))
    end
    return x
end

q_tilde (generic function with 1 method)

In [4]:
a_o = 2
b_o = 0.1
n = 100
s = 10000
N,M = generate(a_o, b_o, s)
Y = rand(Normal(mean(N), 1), n);

In [5]:
function sample()
    Ns = []
    append!(Ns, mean(N))
    Ms = []
    append!(Ms, mean(M))
    
    for i in 2:s
        r = q_tilde(a_o, b_o, n, N[i], M[i], Y)/q_tilde(a_o, b_o, n, N[i-1], M[i-1], Y)
        u = rand(Uniform(0,1))
        if u < min(1, r)
#             append new values
            append!(Ns, N[i])
            append!(Ms, M[i])
        else
#             keep the old values
            append!(Ns, N[i-1])
            append!(Ms, M[i-1])
        end
    end
    return Ns, Ms
end

sample (generic function with 1 method)

In [6]:
Ns, Ms = sample();

In [7]:
mean(N)

2.105575122613332

In [8]:
mean(Ns)

2.1347764312937323

In [9]:
mean(M)

2.6757831142157626

In [10]:
mean(Ms)

2.995318720309996

The mean values of $\mu$ and $\nu$ as in the original case, and after sampling, are close.